### Import necessary modules

In [1]:
import pandas as pd
from collections import Counter
import re
import string
from tqdm import tqdm
#from transformers import pipeline

### Load your dataset

In [2]:
df = pd.read_excel("QA_final.xlsx")  # <-- change to your file
df.drop(columns='answers_start', inplace=True, errors='ignore')  # Remove if not needed

### Split rows with impossible questions

In [3]:
df = df[df['is_impossible'] == False].copy()
df.drop(columns='is_impossible', inplace=True, errors='ignore')  # Remove if not needed
df


,title,context,question,answers
0,"Charleston,_South_Carolina",১৭৭০ সালে ইংল্যান্ডের দ্বিতীয় রাজা চার্লসের স...,চার্লসটন কোন সালে প্রতিষ্ঠিত হয়েছিল?,['১৭৭০ ']
1,"Charleston,_South_Carolina",১৭৭০ সালে ইংল্যান্ডের দ্বিতীয় রাজা চার্লসের স...,চার্লস্টনের আসল নাম কী ছিল?,['চার্লস টাউন']
2,"Charleston,_South_Carolina",১৭৭০ সালে ইংল্যান্ডের দ্বিতীয় রাজা চার্লসের স...,চার্লস টাউনটির নামকরণ করা হয়েছিল কোন রাজার না...,['রাজা চার্লস দ্বিতীয়']
3,"Charleston,_South_Carolina",১৭৭০ সালে ইংল্যান্ডের দ্বিতীয় রাজা চার্লসের স...,শহরটি মূলত কোথায় ছিল?,['অ্যাসলে নদীর পশ্চিম তীরে আলবেমারল পয়েন্ট']
4,"Charleston,_South_Carolina",১৭৭০ সালে ইংল্যান্ডের দ্বিতীয় রাজা চার্লসের স...,২০১০ সালে চার্লস্টনের জনসংখ্যা কত ছিল?,"['১২০,০৮৩']"
...,...,...,...,...
82,self,আমি ১৮ বছরের ছেলে। আমি গল্পের বই পড়তে ভালোবাস...,আমি কি করে দেহচর্যা করি?,['যোগব্যায়ামের মাধ্যমে']
83,self,আমি ১৮ বছরের ছেলে। আমি গল্পের বই পড়তে ভালোবাস...,আমি সপ্তাহে কত বার সাইকেল চালাই?,[' প্রায় চারবার']
84,self,আমি ১৮ বছরের ছেলে। আমি গল্পের বই পড়তে ভালোবাস...,আমি আমার কি কি কাজের মধ্যে ভারসাম্য বজায় রাখা...,['পড়াশোনা এবং অন্যান্য কার্যকলাপ']
85,self,আমি ১৮ বছরের ছেলে। আমি গল্পের বই পড়তে ভালোবাস...,আমি আমার পড়াশোনা এবং কোন কাজের মধ্যে ভারসাম্য...,['অন্যান্য কার্যকলাপ']


### Define text normalization function

In [4]:
import re
import string

def normalize_text(s):
    def remove_punc(text):
        # Define Bengali and English punctuation
        bengali_punc = "।॥“”‘’–—…"
        all_punc = set(string.punctuation + bengali_punc)
        return ''.join(ch for ch in text if ch not in all_punc)

    def white_space_fix(text):
        return ' '.join(text.split())

    return white_space_fix(remove_punc(s.strip()))



### Define Exact Match function

In [5]:
def compute_exact(a_gold, a_pred):
    return int(normalize_text(a_gold) == normalize_text(a_pred))

### Define F1 function

In [6]:
def compute_f1(a_gold, a_pred):
    gold_tokens = normalize_text(a_gold).split()
    pred_tokens = normalize_text(a_pred).split()
    common = Counter(gold_tokens) & Counter(pred_tokens)
    num_same = sum(common.values())
    if len(gold_tokens) == 0 or len(pred_tokens) == 0:
        return (int(gold_tokens == pred_tokens), num_same)
    if num_same == 0:
        return (0, num_same)
    precision = num_same / len(pred_tokens)
    recall = num_same / len(gold_tokens)
    f1 = 2 * precision * recall / (precision + recall)
    return (f1, num_same)


### Load QA model

In [ ]:
model_options=["doerig/banglabert"]

# "Nous-Hermes-2-Mistral-7B-DPO.i1-Q4_K_M.gguf","llama-3-8b-bangla-GGUF-Q4_K_M-unsloth.Q4_K_M.gguf",
# "Nous-Hermes-2-Mistral-7B-DPO.i1-Q4_K_M.gguf": "Nous_Hermes_Mistral_7B",
#           "llama-3-8b-bangla-GGUF-Q4_K_M-unsloth.Q4_K_M.gguf": "Bangla_Llama_3_8B",
model_fn={
          "doerig/banglabert": Bangla_BERT_QA}

#### Model 1:

In [8]:
def Nous_Hermes_Mistral_7B(question, context):
    from nous_Mistral import run
    return run(question, context)

#### Model 2:

In [9]:
def Bangla_Llama_3_8B(question, context):
    from bangla_llama import run
    return run(question, context)

#### Model 3:

In [22]:
def Bangla_BERT_QA(question, context):
    from bangla_bert_qa import run
    return run(question, context)

### Prepare new columns in DataFrame

In [11]:
# Dictionary to hold results per model
results_by_model = {model_name: [] for model_name in model_options}

In [15]:
#Don't run
df["model_name"] = ""
df["generated_answer"] = ""
df["exact_match"] = 0.0
df["f1_score"] = 0.0
df["no_of_words_in_generated_answer"] = 0
df["no_of_words_matched"] = 0
df["framed_sentence"] = ""

### Evaluate

#### Store in separate dataframes

In [ ]:
from tqdm import tqdm



# Iterate over each row in the original dataframe
for _, row in tqdm(df.iterrows(), total=len(df)):
    context = row["context"]
    question = row["question"]
    gold_answers = eval(row["answers"])  # Convert string to list

    for model_name in model_options:
        model_function = model_fn[model_name]
        pred_answer, pred_sentence = model_function(question=question, context=context)
        pred_tokens = normalize_text(pred_answer).split()

        em_scores = []
        f1_scores = []
        word_matches = []

        for gold in gold_answers:
            em = compute_exact(gold, pred_answer)
            f1, n_match = compute_f1(gold, pred_answer)
            em_scores.append(em)
            f1_scores.append(f1)
            word_matches.append(n_match)

        best_em = max(em_scores)
        best_f1 = max(f1_scores)
        best_match_count = max(word_matches)

        results_by_model[model_name].append({
            "context": context,
            "question": question,
            "gold_answers": gold_answers,
            "generated_answer": pred_answer,
            "framed_sentence": pred_sentence,
            "exact_match": best_em,
            "f1_score": best_f1,
            "no_of_words_in_generated_answer": len(pred_tokens),
            "no_of_words_matched": best_match_count,
        })

# Convert each model's result list into a DataFrame
model_dfs = {model_name: pd.DataFrame(results) for model_name, results in results_by_model.items()}


#### Store in a single dataframe 
(Don't run)

In [25]:
# Evaluate
for idx, row in tqdm(df.iterrows(), total=len(df)):
    context = row["context"]
    question = row["question"]
    gold_answers = eval(row["answers"])  # Convert string representation of list to list
     
    for model_name in model_options:
        model_function = model_fn[model_name] 
        pred_answer,pred_sentence= model_function(question=question, context=context)
        pred_tokens = normalize_text(pred_answer).split()
        
        em_scores = []
        f1_scores = []
        word_matches = []

        for gold in gold_answers:
            em = compute_exact(gold, pred_answer)
            f1, n_match = compute_f1(gold, pred_answer)
            em_scores.append(em)
            f1_scores.append(f1)
            word_matches.append(n_match)
        
        # Best match among ground truths
        best_em = max(em_scores)
        best_f1 = max(f1_scores)
        best_match_count = max(word_matches)

    
        # Save results
        df.at[idx, "generated_answer"] = pred_answer
        df.at[idx, "exact_match"] = best_em
        df.at[idx, "f1_score"] = best_f1
        df.at[idx, "no_of_words_in_generated_answer"] = len(pred_tokens)
        df.at[idx, "no_of_words_matched"] = best_match_count
        df.at[idx, "framed_sentence"] = pred_sentence



  0%|          | 0/86 [00:00<?, ?it/s]e:\AnacondaSystem\envs\ideas\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Answer: কুপার


  1%|          | 1/86 [00:19<27:04, 19.11s/it]

Answer: ১৭৭০


  2%|▏         | 2/86 [00:21<12:57,  9.25s/it]

...

Answer: ১৭৭০ সালে


  8%|▊         | 7/86 [00:30<03:14,  2.46s/it]

Answer: চার্লস টাউন


  8%|▊         | 7/86 [00:30<05:48,  4.41s/it]


KeyboardInterrupt: 

### Save the files

In [ ]:
for model_name in model_options:
    results_df = pd.DataFrame(results_by_model[model_name])
    results_df.to_excel(f"{results_by_model[model_name]}.xlsx", index=False)

    print(f"Evaluation complete! Files saved for: {model_name}")

Evaluation complete! Files saved: set1.xlsx


In [ ]:
for model_name in model_options:
    results_df = pd.DataFrame(results_by_model[model_name])
    print(results_df.head())

,context,question,is_impossible,answers,generated_answer,framed_sentence,exact_match,f1_score,no_of_words_in_generated_answer,no_of_words_matched
0,চার্লসটন আমেরিকা যুক্তরাষ্ট্রের দক্ষিণ ক্যারোল...,দক্ষিণ ক্যারোলিনার প্রাচীনতম শহরটি কী?,False,['চার্লসটন'],,,0.0,0.0,0,0
1,চার্লসটন আমেরিকা যুক্তরাষ্ট্রের দক্ষিণ ক্যারোল...,"চার্লসটন, দক্ষিণ ক্যারোলিনা কোন কাউন্টিতে অবস্...",False,['চার্লসটন কাউন্টি '],,,0.0,0.0,0,0
2,চার্লসটন আমেরিকা যুক্তরাষ্ট্রের দক্ষিণ ক্যারোল...,চার্লসটন কোন বন্দরে অবস্থিত?,False,['চার্লসটন '],,,0.0,0.0,0,0
3,চার্লসটন আমেরিকা যুক্তরাষ্ট্রের দক্ষিণ ক্যারোল...,চার্লসটন হারবার কোন মহাসাগরের খাঁড়ি?,False,['আটলান্টিক মহাসাগরের '],,,0.0,0.0,0,0
4,চার্লসটন আমেরিকা যুক্তরাষ্ট্রের দক্ষিণ ক্যারোল...,কুপার নদীর সাথে কোন নদী মিশে গিয়ে চার্লস্টন হ...,False,['অ্যাশলে'],,,0.0,0.0,0,0
5,চার্লসটন আমেরিকা যুক্তরাষ্ট্রের দক্ষিণ ক্যারোল...,চার্লসটন হারবার কোন সমুদ্রের উপর গঠিত?,False,['আটলান্টিক মহাসাগরের '],,,0.0,0.0,0,0
6,চার্লসটন আমেরিকা যুক্তরাষ্ট্রের দক্ষিণ ক্যারোল...,চার্লসটন হারবার থেকে অ্যাশলে নদীর সাথে কোন নদী...,False,['কুপার নদীর '],,,0.0,0.0,0,0
7,চার্লসটন আমেরিকা যুক্তরাষ্ট্রের দক্ষিণ ক্যারোল...,সামারভিলে মেট্রোপলিটন স্ট্যাটিস্টিকাল এরিয়ার ...,False,['নর্থ চার্লসটন'],,,0.0,0.0,0,0
8,চার্লসটন আমেরিকা যুক্তরাষ্ট্রের দক্ষিণ ক্যারোল...,"চার্লসটন, নর্থ ক্যারোলিনা কোন কাউন্টিতে অবস্থি...",False,['চার্লসটন কাউন্টি'],,,0.0,0.0,0,0
